# EDA - Fraud Detection II

<!-- <p align="center">
<img src="../assets/illustration_eda.png" alt="drawing" width="800"/>
<p> -->


In [ ]:
# prelude

import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
import plotly.express as px
import matplotlib.pyplot as plt

from pathlib import Path
from datetime import datetime, timedelta



k_AssetsDir     = "../data/"
k_FileName      = "fraud_test.csv"
k_Gold          = 1.618     # gold number for ratio
k_Width         = 12
k_Height        = k_Width/k_Gold
k_random_state  = 42           
k_test_size     = 20/100        


In [ ]:
# -----------------------------------------------------------------------------
filename_in = Path(k_AssetsDir)/k_FileName
df = pd.read_csv(filename_in)

# Alternative (AWS S3 bucket)
# df = pd.read_csv("https://lead-program-assets.s3.eu-west-3.amazonaws.com/M05-Projects/fraudTest.csv")

df.columns = df.columns.str.lower()
df.head(3)

In [ ]:
# -----------------------------------------------------------------------------
def quick_View(df):
  summary_lst = []
  
  for col_name in df.columns:
    col_dtype               = df[col_name].dtype
    num_of_null             = df[col_name].isnull().sum()
    percent_of_null         = num_of_null/len(df)
    num_of_non_null         = df[col_name].notnull().sum()
    num_of_distinct_values  = df[col_name].nunique()
    
    if num_of_distinct_values <= 10:
        distinct_values_counts = df[col_name].value_counts().to_dict()
    else:
        top_10_values_counts    = df[col_name].value_counts().head(10).to_dict()
        distinct_values_counts  = {k: v for k, v in sorted(top_10_values_counts.items(), key=lambda item: item[1], reverse=True)}

    if col_dtype != "object":
       max_of_col = df[col_name].max()
       min_of_col = df[col_name].min()
       outlier_hi = df[col_name].mean() + 3*df[col_name].std()
       outlier_lo = df[col_name].mean() - 3*df[col_name].std()
    else:
       max_of_col = -1
       min_of_col =  1
       outlier_hi = -1
       outlier_lo =  1
    
    summary_lst.append({
      "name"                : col_name,
      "dtype"               : col_dtype,
      "# null"              : num_of_null,
      "% null"              : (100*percent_of_null).round(2),
      "# NOT null"          : num_of_non_null,
      "distinct val"        : num_of_distinct_values,
      "-3*sig"              : round(outlier_lo,2) ,
      "min"                 : round(min_of_col,2),
      "max"                 : round(max_of_col,2),
      "+3*sig"              : round(outlier_hi,2) ,
      "distinct val count"  : distinct_values_counts
    })
  
  tmp_df = pd.DataFrame(summary_lst)
  return tmp_df

In [ ]:
tmp_df = quick_View(df)
display(tmp_df.sort_values(by="# null", ascending=False))                 

### <span style="color:orange"><b>Comments :</b></span>

Le tableau ci-dessus montre que : 

* Il y a 23 features (0 à 22)
* La première est un index
* Aucune feature ne comporte de valeur nulle (voir les colonnes # null, % null et # NOT null)
* Pas de soucis dans l'index. Il y a 555_719 ligne avec des index différents
* Rien de très remarquable à part la très grande disparité entre les nombres de transactions frauduleuses (2145) et légitimes (553_574)
* Dans le dataset il y a donc **0.38%** de transactions frauduleuses ($\frac{2145}{(553574+2145)}$)
* Compte tenu du contexte (détection de fraudes à la carte bancaire) il est décidé de porter une attention particulière au taux de faux négatif. C'est le taux de transaction frauduleuses qui seront classées comme légitimes. On souhaite que cette valeur soit minimale. Il est très important d'en détecter un maximum même si certaines transactions légitimes sont classées comme frauduleuses.
* En plus du taux de faux négatif, suite à l'entrainement on sauvegardera sur le serveur mlflow tracking (https://fraud-202406-70e02a9739f2.herokuapp.com/) la matrice de confusion, la ROC curve etc.

### <span style="color:orange"><b>Conclusion :</b></span>
* **0.38%** : le taux de transactions frauduleuse dans le jeu de test
* Métrique principale : Taux de faux négatifs


### <span style="color:orange"><b>Carte :</b></span>

In [ ]:
df_fraud = df[df["is_fraud"] == 1]
fig = px.scatter_mapbox(df_fraud, lat="merch_lat", lon="merch_long", zoom=3,width=1200, height=800)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(title="Localisation des fraudes (train dataset)")
fig.show()
